# Redux
Code for this tutorial https://github.com/yilengyao/parsnip

Allows you to store your entire application state in a single object using pure functions

Actions
- Actions are objects that describe an event such as `CREATE_TASK`.
- Actions must be dispatched to apply any updates.

Fundamental Concept:
- `Action`: An object that describes an event. The phrase synchronous action always refers to these action objects. If a synchronous action is dispatched, it's received by the store immediately. Actions have a required `type` peroperty and can optionally have additional fields that stroe data needed to handle the action:
```
{
    type: 'FETCH_TASKS_SUCCEEDED',
    payload: {
        tasks: [...]
    }
}
```
- `Action creator`: A function that returns an action.
- `Synchronous action creator`: All action creators that return an action are considerred synchronous action creators:
```
function fetchTasksSucceeded(tasks) {
    return {
        type: 'FETCH_TASKS_SUCCEEDED',
        payload: {
            tasks: [...]
        }
    }
}
```
- `Async action creator`: An action creator that does contain async code (the most common example being network requests). They typically make one or more API calls and dispatch one or more actions of certain points in the request's lifecycle. Often, instead of retuning an action directly, they may return synchronous action creators for readability:
```
export function fetchTasks() {
    return dispatch => {
        api.fetchTasks().then(resp => {
            dispatch(fetchTasksSucceeded(resp.data));
        });
    };
}
```

or using async/await
```
export function fetchTasks() {
    return async dispatch => {
        const resp = await api.fetchTasks();
        dispatch(fetchTasksSucceeded(resp.data));
    };
}
```

# Asynchronous actions
Asynchronous actions usually include
- One or more side effects, such as an AJAX request.
- One or more synchronous dispatcehs, such as dispatching an action after an AJAX request has resolved.

## Redux thunk
Redux thunk allows your action creator to return a function without thunk, your action creator is only allowed to return an action object.

Redux by default, action creators are expected to return plain action objects-simple JavaScript objects with a `type` property and, optionally, a `payload`. However, `Redux Thunk` is a middleware that extends Redux's capabilities by allowing action creators to return functions instead of plain action objects.

Redux thunk is a middleware for Redux that allows you to write action creators that return a funciton instead of an action object. It enables writing asynchronous logic that interacts with the Redux store.

How Does Redux Thunk Work?
- Intercepts Functions: when Redux Thunk detects that an action cretor returns a function, it intercepts that function.
- Provides `dispatch` and `getState`: The return funciton receives `dispatch` and `getState` as arguments.
- Allows Side Effects: Inside the funciton, you can perform asynchronous operations, side effects, or dispatch multiple actions.

Redux Thunk enables:
1. Asynchronous Actions: It allows you to write action creators that handle async logoc, such as API calls.
2. Conditional Dispatching: You can dispatch actions conditionally based on the current state.
3. Complex Action Logic: Enables you to sequence multiple dispatches and perform more complex action logic.

### Thunk provides the following functions to your action creator
#### 1. `dispatch`
A function that llows you to dispatch actions. You can dispatch synchronous actions or other asynchronous thunks.
Ex.
```
dispatch({ type: 'ACTION_TYPE', payload: data });
```

#### 2. `getState`
A function that retruns the current satte of the Redux store. Useful for making decisions based on the current state before dispatching actions. Ex.
```
const state = getState();
```

#### 3. `extraArgument` (optional)
If you configure Redux Thunk with extra arguments, they are passed to your thunk actions. Allows injection of custom dependencies like API clients or other utilities.
Ex.
```
const store = configureStore(
    rootReducer,
    applyMiddleware(thunk.withExtraArgument(api))
);
```
Access in thunk action
```
return (dispatch, getState, api) => {
    // Use api here
};
```

#### Example
index.js
```
import React from 'react';
import ReactDOM from 'react-dom/client';
import App from './App';
import { Provider } from 'react-redux';
import { configureStore } from '@reduxjs/toolkit';
import tasksReducer from './reducers';
import './index.css';

const store = configureStore({
  reducer: {
    tasks: tasksReducer,
  },
});

const root = ReactDOM.createRoot(document.getElementById('root'));

root.render(
  <Provider store={store}>
    <App />
  </Provider>
);
```

App.js
```
import React, { Component } from 'react';
import { connect } from 'react-redux';
import TasksPage from './components/TasksPage';
import { createTask, editTask, fetchTasks } from './actions';
import FlashMessage from './components/FlashMessage';

class App extends Component {
  componentDidMount() {
    this.props.dispatch(fetchTasks());
  }

  onCreateTask = ({ title, description }) => {
    this.props.dispatch(createTask({ title, description }));
  };

  onStatusChange = (id, status) => {
    this.props.dispatch(editTask(id, { status }))
  }

  render() {
    return (
      <div className="container">
        {this.props.error &&
          <FlashMessage message={this.props.error} />}
        <div className="main-content">
          <TasksPage
            tasks={this.props.tasks.tasks}
            onCreateTask={this.onCreateTask}
            onStatusChange={this.onStatusChange}
            isLoading={this.props.isLoading}
          />
        </div>
      </div>
    );
  }
}

function mapStateToProps(state) {
  const { tasks, isLoading, error } = state.tasks;
  return {
    tasks,
    isLoading,
    error
  };
}

export default connect(mapStateToProps)(App);
```

actions/index.js
```
export function editTask(id, params = {}) {
  return (dispatch, getState) => {
    const task = getTaskById(getState().tasks, id);
    const updatedTask = Object.assign({}, task, params);

    api.editTask(id, updatedTask).then(resp => {
      dispatch(editTaskSucceeded(resp.data));
    })
      .catch(err => {
        dispatch(fetchTasksFailed(err.message + " edit task"));
      });
  };
}
```


## View and Server Actions
Typically you have 2 entities that can modify application state: users and servers. Actions can be divided into 2 groups, one for each actor: view actions and server actions.
- `View Actions`: Initiated by users. Think `FETCH_TASKS`, `CREATE_TASK`, and `EDIT_TASK`. For example, a user clicks a button and an action is dispatched.
- `Server Actions`: Initiated by the server. For example, a request successfully completes, and an action is dispatched with the response. When you implemented fetching tasks via AJAX, you introduce your first server action: `FETCH_TASKS_SUCCEEDED`.

## Request Lifecycle
In are network requests, there are 2 moments in time that you care about: when the request starts, and when it completes. If you model these events as actions, you end up with 3 distinct action types that help describe the request-response lifecycle. Using fetching tasks as an example, note the following are the 3 action types:
- `FETCH_TASKS_STARTED`: Dispatch when a request is initiated. Typically used to render a loading indicator.
- `FETCH_TASKS_SUCCEEDED`: Dispatcheed when a request is completed successfully. Takes data from the response body and loads it into the store.
- `FETCH_TASKS_FAILED`: Dispatched when arequest fails for any reason, such as network failure or a response with a non-200 status code. Payloads often incluade an error messate from the server.


# Redux Saga
Redux Saga is a middleware libirary that helps manage complex asynchronous operations in Redux applications. It allos you to handle side effects (like data fetching, impure actions) in a more manageable and scalable way.

It aims to make side effects (asynchronous things like dat fetching and impire things like accessing the browser cache) easier to manage, more efficient to execute, and better at handling failures.

## How Redux Saga Works?
Redux Saga uses generator functions to create "sagas" that can pause (`yield`) and resume, allowing you to write asynchronous code that looks synchronous.
- `Generator Function`: Special functions that can pause execution and resume at a later point.
- `Effects`: Plain JavaScript objects that contain instrusctions for the middleware on what to do (eg., dispatch an action, make an API call).
- `Middleware`: Redux Sata runs alongside your Redux store, intercepting dispatched actions and running sagas in response.

## Key Concepts
1. Sagas: A saga is a generator function that yilds object to the Redux Saga middleware.
```
import { call, put, takeEvery } from 'redux-saga/effects';

// Worker saga
function* fetchUser(action) {
    try {
        const user = yield call(Api.fetchUser, action.payload.userId);
        yield put({ type: 'USER_FETCH_SUCCEEDED', user: user });
    } catch (e) {
        yield put({ type: 'USER_FETCH_FAILED', message: e.message });
    }
}

// Watcher saga
function* watchFetcher() {
    yield takeEvery('USER_FETCH_REQUESTED', fetchUser);
}
```

2. Effects
- - `call`: Invokes a function and waits for it to complete.
```
const rersult = yield call(Api.fetchData, params);
```

- - `put`: Dispatches an action to the Redux store.
```
yield put({ type: 'ACTION_TYPE', payload });
```

- - `takeEvery`: Listens for dispatch action and runs a saga each time.
```
yield takeEvery(`ACTION_TYPE`, sagaFunction);
```

- - `takeLatest`: Runs only the latest saga if multiple actions are dispatched quickly.
```
yield takeLatest('ACTION_TYPE', sagaFunction);
```

3. Middleware Stup
Installing Redux Saga:
```
npm install redux-saga
```

Apply Middleware
```
import  createStore, applyMiddleware } from 'redux';
import createSagaMiddleware from 'redux-saga';
import rootReducer from `./reducers';
import rootSaga from './sagas';

const sagaMiddleware = createSagaMiddleware();
const store = createStore(rootReducer, applyMiddleware(sagaMiddleware));

sagaMiddleware.run(rootSaga);
```

# Selectors
Selectors are pure functions that extract, derive, or compute data from the Redux store's state. They encapsulate the logic for accessing and transforming the state, providing a consistent and reusable way to retrieve data throughout your application.

Why Use Selectors?
- Encapsulation: Keep state acess logic in one place.
- Reusability: Use the same selector in multiple components or parts of your application.
- Maintainability: Centralize state query logic, making it easeir to update if the state shape changes.
- Performanace Optimization: Memoized selectors can prevent unnecessary recalculations, improving performance.

## Basic Selector Example
Redux state
```
const state = {
    users: {
        byId: {
            '1': { id: '1', name: 'Alice' },
            '2': { id: '2', name: 'Bob' }
        },
        allIds: ['1', '2']
    }
};
```

A selector to get all users
```
const selectAllUsers = (state) => {
    return state.users.allIds.map(id => state.users.byId[id]);
};
```

Use in a component
```
import { useSelector } from 'react-redux';

function UserList() {
  const users = useSelector(selectAllUsers);

  return (
    <ul>
      {users.map(user => (
        <li key={user.id}>{user.name}</li>
      ))}
    </ul>
  );
}
```

## Memoized selectors with Reslect
Reselect is a library commonly used with Redux to create memoized selectors. Memoization caches the results of function callsl and returns the cached result when the same inputs occur again, avoiding unnecessary recalculations.

Installation
```
npm install reselect
```

Creating Memoized Selectors
```
import { createSelector } from 'reselect';

const selectUsersById = (state) => state.users.byId;
const selectUserIds = (state) => state.users.allIds;

const selectAllUsers = createSelector(
    [selectusersById, selectUserIds],
    (usersById, useIds) => userIds.map(id => usersById[id])
);
```
- `selectUsesById` and `selectUserIds` are input selectors that simply extract parts of the state.
- `selectAllUsers` is a memoized selector that depends on these inputs.
- The result is recumputed only when the input change.

## Caching
When using `Reselect`, memoization works by caching the inputs (query) and the outputs (response) of the selector function.

Memorization works only if the data is immutable. If data is mutated in place without changing references, Reselect may not detect changes leading to incorrect state results.


## `createSelector`
```
const selector = createSelector(
    [...inputSelectors],
    resultFunction
);
```
- Input Selectors: Functions that extract specific pieces of data from the Redux state, They server as the input sources for data needed in computations.
- Result Function: A function that takes the output of the input selectors and computes the final derived data.

The `createSelector` creates a selector function that caches the output, returning the cached result when called with the same inputs again.